In [4]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_recall_fscore_support
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

SOTAB CTA

In [5]:
sotab_cta_train = pd.read_csv('SOTAB CTA/sotab_v2_cta_training_set.csv')
sotab_cta_test = pd.read_csv('SOTAB CTA/sample_test.csv')

In [6]:
sotab_cta_test

,table_name,column_index,label
0,Event_blugrottonj.com_September2020_CTA.json.gz,4,Place/name
1,Event_blugrottonj.com_September2020_CTA.json.gz,5,telephone
2,LocalBusiness_basellive.ch_September2020_CTA.j...,2,openingHours
3,LocalBusiness_worldcement.com_September2020_CT...,5,Country
4,Movie_321movies.org_September2020_CTA.json.gz,10,Rating
...,...,...,...
313,Product_babyartikelcheck.de_September2020_CTA....,0,Product/name
314,Product_divas-club.de_September2020_CTA.json.gz,0,Product/name
315,Product_divas-club.de_September2020_CTA.json.gz,1,Product/description
316,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber


In [7]:
final_test_table=pd.DataFrame()
table=sotab_cta_test
primary_path='SOTAB CTA/Test/'


all_data_from_all_columns = []
all_target = []
for table_index, table_values in table.iterrows():

    full_path = primary_path + table_values[0]
    index_of_desired_columns = table_values[1]
    label_of_table = table_values[2]


    df = pd.read_json(full_path, compression='gzip', lines=True) 
    #print(table_values[0])
    all_values_from_desired_columns = []

    #print(table_values[0])
    #print(index_of_desired_columns)
    for value in df[index_of_desired_columns]:
        if value != None:
            all_values_from_desired_columns.append(value)

    joined_values = ', '.join(map(str, all_values_from_desired_columns))

    all_data_from_all_columns.append(joined_values)
    #all_data_from_all_columns.append(all_values_from_desired_columns)
    all_target.append(label_of_table)

final_test_table['data']=all_data_from_all_columns
final_test_table['target']=all_target


In [8]:
final_test_table

,data,target
0,"Blu Grotto, Blu Grotto",Place/name
1,"(732) 571-7900, (732) 571-7900",telephone
2,"Th 12:00-18:30, Fr 12:00-18:30, Sa 11:00-17:00...",openingHours
3,"United Kingdom, United Kingdom, United Kingdom...",Country
4,"R, R, PG, PG-13, R, R, R, PG-13, PG, R, PG, PG...",Rating
...,...,...
313,"Beurer BY 88 Smart, Qtus Tody, ABC Design Sals...",Product/name
314,Transparent clubwear bodysuit made of tulle an...,Product/name
315,"Material: 62% Polyester, 34% Polyamid, 3% Elas...",Product/description
316,"(303) 996-7851, (303) 996-7851, (303) 996-7851...",faxNumber


In [9]:
final_train_table=pd.DataFrame()
table=sotab_cta_train
primary_path='SOTAB CTA/Train/'


all_data_from_all_columns = []
all_target = []
for table_index,table_values in table.iterrows():

    full_path = primary_path + table_values[0]
    index_of_desired_columns = table_values[1]
    label_of_table = table_values[2]


    df = pd.read_json(full_path, compression='gzip', lines=True) 
    #print(table_values[0])
    all_values_from_desired_columns = []

    #print(table_values[0])
    #print(index_of_desired_columns)
    for value in df[index_of_desired_columns]:
        if value != None:
            all_values_from_desired_columns.append(value)

    joined_values = ', '.join(map(str, all_values_from_desired_columns))

    all_data_from_all_columns.append(joined_values)        
    all_target.append(label_of_table)

final_train_table['data']=all_data_from_all_columns
final_train_table['target']=all_target

In [12]:
final_train_table

,data,target
0,"2020-07-10, 2016-04-08, 2013-09-13, 2016-08-05...",Date
1,"The Sleep Revolution, Viva, Ame, Lidere, The S...",Book/name
2,"en-US, pt-BR, pt-BR, pt-BR, pt-BR, pt-BR, pt-B...",Language
3,"Homer William Bedell Stanford W. Stanford, Dav...",Person/name
4,"Paperback, Hardcover, Paperback, None, Paperba...",BookFormatType
...,...,...
116882,"Billions, Bonanza, Voltron: Legendary Defender...",CreativeWorkSeries
116883,"Family Guy Season 2 Episode 3, Naked and Afrai...",TVEpisode/name
116884,"Family Guy, Naked and Afraid XL, Animal Kingdo...",CreativeWorkSeries
116885,"Psych Season 2 Episode 8, Still a Mystery Seas...",TVEpisode/name


In [13]:
types = list(final_train_table["target"].unique())
 
final_train_table['encoded_label'] = final_train_table['target'].apply(lambda row: types.index(row))
final_test_table['encoded_label'] = final_test_table['target'].apply(lambda row: types.index(row))

In [14]:
len(final_test_table.encoded_label.unique())

82

In [15]:
final_train_table

,data,target,encoded_label
0,"2020-07-10, 2016-04-08, 2013-09-13, 2016-08-05...",Date,0
1,"The Sleep Revolution, Viva, Ame, Lidere, The S...",Book/name,1
2,"en-US, pt-BR, pt-BR, pt-BR, pt-BR, pt-BR, pt-B...",Language,2
3,"Homer William Bedell Stanford W. Stanford, Dav...",Person/name,3
4,"Paperback, Hardcover, Paperback, None, Paperba...",BookFormatType,4
...,...,...,...
116882,"Billions, Bonanza, Voltron: Legendary Defender...",CreativeWorkSeries,81
116883,"Family Guy Season 2 Episode 3, Naked and Afrai...",TVEpisode/name,80
116884,"Family Guy, Naked and Afraid XL, Animal Kingdo...",CreativeWorkSeries,81
116885,"Psych Season 2 Episode 8, Still a Mystery Seas...",TVEpisode/name,80


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=10)
X_train = tfidf.fit_transform(final_train_table['data']) #.toarray()
X_test = tfidf.transform(final_test_table['data']) #.toarray()

X_test.shape

(318, 115296)

In [20]:
X_train.shape

(116887, 115296)

In [21]:
values = list(final_train_table['encoded_label'])
n_values = np.max(values) + 1
y_train = np.eye(n_values)[values]
y_train.shape

(116887, 82)

In [22]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [23]:
values = list(final_test_table['encoded_label'])
y_test = np.eye(n_values)[values]
y_test.shape

(318, 82)

In [26]:
feature_names = tfidf.get_feature_names_out()
print("Number of features:", len(feature_names))
print("Some example features:", feature_names[:10])

Number of features: 115296
Some example features: ['00' '000' '0000' '00000' '000000' '00001' '0001' '0001pt' '0002' '0003']


In [18]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiClass',
    verbose=False
)
model.fit(X_train, np.argmax(y_train, axis=1))

In [17]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
	iterations=100, depth=6, learning_rate=0.1, 
loss_function='MultiClass', verbose=False)

model.fit(X_train, y_train)

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/data_providers.cpp:612: Currently only multi-regression, multilabel and survival objectives work with multidimensional target

In [64]:
from catboost import CatBoostClassifier

models = []
for col in range(y_train.shape[1]):
    model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=False)
    model.fit(X_train, y_train[:, col])
    models.append(model)

CatBoostError: bad allocation

In [27]:
model = CatBoostClassifier(
    loss_function='MultiLogloss',
    learning_rate=0.1,
    iterations=7
)
model.fit(X_train, y_train)

0:	learn: 0.4830941	total: 39m 40s	remaining: 3h 58m 2s
1:	learn: 0.3446185	total: 1h 20m 6s	remaining: 3h 20m 15s
2:	learn: 0.2542071	total: 1h 56m 33s	remaining: 2h 35m 25s
3:	learn: 0.1946845	total: 2h 33m 19s	remaining: 1h 54m 59s
4:	learn: 0.1552612	total: 3h 9m 11s	remaining: 1h 15m 40s
5:	learn: 0.1268151	total: 3h 45m 11s	remaining: 37m 31s
6:	learn: 0.1047389	total: 4h 21m 15s	remaining: 0us


In [28]:
y_pred = model.predict(X_test)

In [29]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00         7
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         4
          13       0.00      0.00      0.00         4
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         3
    

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test, y_pred, average="macro")

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
macro

0.0

In [32]:
micro_f1 = f1_score(y_test, y_pred, average="micro")

In [33]:
print(micro_f1)

0.0


In [34]:
len(y_pred)

318

In [35]:
y_pred[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
import pickle
with open('CatBoost Evaluation model/sotab-CTA-model.pkl', 'wb') as file:
    pickle.dump(model, file)

SOTAB CPA

In [2]:
sotab_cpa_train = pd.read_csv('SOTAB CPA/sotab_v2_cpa_training_set.csv')
sotab_cpa_test = pd.read_csv('SOTAB CPA/sample_test.csv')

In [3]:
sotab_cpa_test

,table_name,main_column_index,column_index,label
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone
...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions


In [4]:
import re
gt = {'train':{}, 'test':{}}
for index, row in sotab_cpa_train.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in sotab_cpa_test.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]
    
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    

    text = re.sub(' +', ' ', str(text)).strip()
    
    return text


def get_all_table_columns(file_name, index):
    

    if file_name in sotab_cpa_train["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    

    cleaned_main="{}{}"

    x = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) 
    cleaned_main=cleaned_main.format("", x)
    cleaned_rows.append(cleaned_main)

    cleaned_rows.append("")
    
    
    for row in df.iloc[:, index].tolist():
        get_values_var=7
        cleaned = " ".join(clean_text(row).split()[:20])
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
        else:
            get_values_var=get_values_var-1

       
    
    return " ".join(cleaned_rows[:get_values_var])

In [5]:
test_examples = []
for table in gt['test']:
    for column in gt['test'][table]:
        col_str = get_all_table_columns(table, column)
        test_examples.append([table, column, col_str, gt['test'][table][column], table.split("_")[0]])

In [6]:
test_examples

[['Book_antipodean.com_September2020_CPA.json.gz',
  4,
  "On Coxalgia, or Hip Disease. Simonds Saws & Knives, Gold Medals and Highest Awards Everywhere. Atlanta 1895. Art Nouveau poster. North Island, New Zealand, Sheet 3, northeast corner from Tauranga to Dannevirke, folding map on linen. Trade Receipts for wine and beer from Perth, Australia merchants 'H. Sherwood & Co., Wine & Spirit Merchants' and 'D. The American Trans-Continental Route via New York Central & Hudson River R. R. and Connections, the Only 4 Track Line.  Collins, Livermore & Knight Co, Marcus F. Marks, Government Printer. Burlington. Visual Anthropology,",
  'publisher',
  'Book'],
 ['Book_antipodean.com_September2020_CPA.json.gz',
  3,
  "On Coxalgia, or Hip Disease. Simonds Saws & Knives, Gold Medals and Highest Awards Everywhere. Atlanta 1895. Art Nouveau poster. North Island, New Zealand, Sheet 3, northeast corner from Tauranga to Dannevirke, folding map on linen. Trade Receipts for wine and beer from Perth, Aus

In [7]:
final_table_sotab_cpa=pd.DataFrame()
labels=[]
values=[]
for i in test_examples:
    labels.append(i[-2])
    values.append(i[2])

final_table_sotab_cpa['data']=values
final_table_sotab_cpa['cpa label']=labels

In [8]:
test_table_sotab_cpa=final_table_sotab_cpa

test_table_sotab_cpa=test_table_sotab_cpa

In [15]:
train_examples = []
for table in gt['train']:
    for column in gt['train'][table]:
        col_str = get_all_table_columns(table, column)
        train_examples.append([table, column, col_str, gt['train'][table][column], table.split("_")[0]])

In [16]:
train_examples

[['Book_11x17.pt_September2020_CPA.json.gz',
  3,
  'A Cidade Perdida A Cúpula - livro 1 Misery A Cúpula - Livro 2 O Olho de Deus  2020-07-10 2016-04-08 2013-09-13 2016-08-05 2019-05-10',
  'datePublished',
  'Book'],
 ['Book_11x17.pt_September2020_CPA.json.gz',
  1,
  'A Cidade Perdida A Cúpula - livro 1 Misery A Cúpula - Livro 2 O Olho de Deus  9789722539739 9789722531924 9789722527118 9789722532457 9789722537636',
  'isbn',
  'Book'],
 ['Book_11x17.pt_September2020_CPA.json.gz',
  2,
  'A Cidade Perdida A Cúpula - livro 1 Misery A Cúpula - Livro 2 O Olho de Deus  728 704 480 656 528',
  'numberOfPages',
  'Book'],
 ['Book_1jour-1jeu.com_September2020_CPA.json.gz',
  9,
  'Escape Game: Junior - La Malédiction de la Momie Escape Game: Serial Killer Escape Game: La Tour Noyée Escape Game: Junior - Opération Pizza EXIT: Le Livre - Journal de Bord 29  0 0 0 0 0',
  'worstRating',
  'Book'],
 ['Book_1jour-1jeu.com_September2020_CPA.json.gz',
  5,
  'Escape Game: Junior - La Malédiction de

In [17]:
final_table_sotab_cpa=pd.DataFrame()
labels=[]
values=[]
for i in train_examples:
    labels.append(i[-2])
    values.append(i[2])

final_table_sotab_cpa['data']=values
final_table_sotab_cpa['cpa label']=labels

In [18]:
train_table_sotab_cpa=final_table_sotab_cpa

train_table_sotab_cpa=train_table_sotab_cpa

In [19]:
test_table_sotab_cpa

,data,cpa label
0,"On Coxalgia, or Hip Disease. Simonds Saws & Kn...",publisher
1,"On Coxalgia, or Hip Disease. Simonds Saws & Kn...",availability
2,"On Coxalgia, or Hip Disease. Simonds Saws & Kn...",price
3,Infant Loss Drop-In 7-9 Evening Drop-In 7-9 ZO...,eventStatus
4,Infant Loss Drop-In 7-9 Evening Drop-In 7-9 ZO...,eventAttendanceMode
...,...,...
504,The Big Fib Season 1 Episode 5 Warrior Nun Sea...,url
505,The Big Fib Season 1 Episode 5 Warrior Nun Sea...,partOfSeries
506,Salva El Tigre Pimemento Jim Carrey Chapter Tw...,episodeNumber
507,Salva El Tigre Pimemento Jim Carrey Chapter Tw...,description


In [20]:
train_table_sotab_cpa

,data,cpa label
0,A Cidade Perdida A Cúpula - livro 1 Misery A C...,datePublished
1,A Cidade Perdida A Cúpula - livro 1 Misery A C...,isbn
2,A Cidade Perdida A Cúpula - livro 1 Misery A C...,numberOfPages
3,Escape Game: Junior - La Malédiction de la Mom...,worstRating
4,Escape Game: Junior - La Malédiction de la Mom...,datePublished
...,...,...
109989,Doom Patrol Patrol Fun Size Patrol What Is and...,datePublished
109990,Billions Season 5 Episode 7 Bonanza Season 6 E...,duration
109991,Billions Season 5 Episode 7 Bonanza Season 6 E...,description
109992,Billions Season 5 Episode 7 Bonanza Season 6 E...,url


In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=5)
X_train_cpa_sotab = tfidf.fit_transform(train_table_sotab_cpa['data']) 
X_test_cpa_sotab = tfidf.transform(test_table_sotab_cpa['data'])

X_test_cpa_sotab.shape

(509, 53085)

In [90]:
X_test_cpa_sotab

<509x53085 sparse matrix of type '<class 'numpy.float64'>'
	with 13011 stored elements in Compressed Sparse Row format>

In [91]:
types = list(train_table_sotab_cpa["cpa label"].unique())
 
train_table_sotab_cpa['encoded_label'] = train_table_sotab_cpa['cpa label'].apply(lambda row: types.index(row))
test_table_sotab_cpa['encoded_label'] = test_table_sotab_cpa['cpa label'].apply(lambda row: types.index(row))

In [62]:
values = list(train_table_sotab_cpa['encoded_label'])
n_values = np.max(values) + 1
y_train_cpa_sotab = np.eye(n_values)[values]
y_train_cpa_sotab.shape

(109994, 108)

In [92]:
values = list(test_table_sotab_cpa['encoded_label'])
n_values = np.max(values) + 1
y_test_cpa_sotab = np.eye(n_values)[values]
y_test_cpa_sotab.shape

(509, 108)

In [94]:
feature_names = tfidf.get_feature_names_out()
print("Number of features:", len(feature_names))
print("Some example features:", feature_names[:10])

Number of features: 53085
Some example features: ['00' '000' '0000' '000000' '0000000' '00000007' '00000008' '0001'
 '0001wyb' '0002']


In [26]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
	iterations=100, depth=6, learning_rate=0.1, 
loss_function='MultiClass', verbose=False)

model.fit(X_train_cpa_sotab, y_train_cpa_sotab)

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/data_providers.cpp:612: Currently only multi-regression, multilabel and survival objectives work with multidimensional target

In [27]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiClass',
    verbose=False
)
model.fit(X_train_cpa_sotab, np.argmax(y_train_cpa_sotab, axis=1))

In [104]:
model_cpa_sotab = CatBoostClassifier(
    loss_function='MultiLogloss',
    iterations=15
)
model_cpa_sotab.fit(X_train_cpa_sotab, y_train_cpa_sotab)

Learning rate set to 0.5
0:	learn: 0.0985117	total: 2m 44s	remaining: 38m 24s
1:	learn: 0.0530093	total: 5m 27s	remaining: 35m 29s
2:	learn: 0.0422695	total: 8m 17s	remaining: 33m 11s
3:	learn: 0.0408244	total: 11m 13s	remaining: 30m 51s
4:	learn: 0.0400531	total: 13m 53s	remaining: 27m 47s
5:	learn: 0.0387722	total: 16m 28s	remaining: 24m 43s
6:	learn: 0.0377302	total: 19m 15s	remaining: 22m
7:	learn: 0.0369922	total: 22m 17s	remaining: 19m 30s
8:	learn: 0.0364593	total: 25m 17s	remaining: 16m 51s
9:	learn: 0.0356567	total: 27m 59s	remaining: 13m 59s
10:	learn: 0.0347965	total: 30m 44s	remaining: 11m 10s
11:	learn: 0.0344388	total: 33m 24s	remaining: 8m 21s
12:	learn: 0.0341176	total: 36m 4s	remaining: 5m 33s
13:	learn: 0.0338072	total: 38m 45s	remaining: 2m 46s
14:	learn: 0.0335582	total: 41m 27s	remaining: 0us


In [105]:
y_pred_cpa_sotab = model_cpa_sotab.predict(X_test_cpa_sotab)

In [106]:
print("Classification Report:")
print(classification_report(y_test_cpa_sotab, y_pred_cpa_sotab))

Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.11      0.15         9
           1       1.00      0.11      0.20         9
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         6
           4       0.86      1.00      0.92         6
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00        12
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         7
          10       0.56      0.42      0.48        12
          11       0.67      0.25      0.36         8
          12       0.00      0.00      0.00         1
          13       1.00      0.57      0.73         7
          14       0.67      0.25      0.36         8
          15       1.00      0.88      0.93         8
          16       1.00      0.33      0.50         6
    

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test_cpa_sotab, y_pred_cpa_sotab, average="macro")

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [108]:
macro

0.1251744624293644

In [109]:
micro_f1 = f1_score(y_test_cpa_sotab, y_pred_cpa_sotab, average="micro")

In [110]:
print(micro_f1)

0.21333333333333332


In [111]:
len(y_pred_cpa_sotab)

509

In [113]:
y_pred_cpa_sotab[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [114]:
import pickle
with open('CatBoost Evaluation model/sotab-CPA-model.pkl', 'wb') as file:
    pickle.dump(model_cpa_sotab, file)

WikiTable CTA MULTI-LABEL CLASSIFIER

In [59]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

In [60]:
cta_train = pd.read_json('WikiData/train.table_col_type.json')
cta_test = pd.read_json('WikiData/test.table_col_type.json')

In [135]:
cta_all_labels = ['soccer.football_league', 'government.government_office_or_title', 'organization.non_profit_organization', 'olympics.olympic_games', 'cvg.cvg_genre', 'ice_hockey.hockey_position', 'tv.tv_network', 'aviation.airline', 'american_football.football_conference', 'soccer.football_world_cup', 'american_football.football_coach', 'military.military_unit', 'military.military_post', 'music.media_format', 'tv.tv_personality', 'baseball.baseball_team', 'cvg.cvg_developer', 'soccer.football_award', 'ice_hockey.hockey_team', 'tv.tv_writer', 'meteorology.tropical_cyclone_season', 'soccer.fifa', 'cvg.cvg_publisher', 'baseball.baseball_player', 'sports.sports_championship', 'soccer.football_team_manager', 'sports.golfer', 'baseball.baseball_position', 'military.rank', 'cvg.cvg_platform', 'music.musical_group', 'amusement_parks.ride', 'music.genre', 'music.lyricist', 'music.record_label', 'meteorology.tropical_cyclone', 'aviation.airport']

In [136]:
final_test_table=pd.DataFrame()
all_values=[]
labelsx=[]
for index_cta, row_cta in cta_test.iterrows():
    

    for table_index,table_row in enumerate(row_cta[6]):
        labels=[]
        for label in  row_cta[7][table_index]:
            if label in cta_all_labels:
                labels.append(label)
        

        if len(labels)>0:
            value_list=[]
            for value_index, value in table_row:
                value_list.append(value[1])
            joined_value_list = ', '.join(map(str, value_list))   
            
            all_values.append(joined_value_list)
            #joined_label=', '.join(map(str, row_cta[7][table_index]))
            joined_label=labels
            labelsx.append(joined_label)

final_test_table['data']=all_values
final_test_table['Target']=labelsx

print(len(all_values))
print(len(labelsx))

807
807


In [137]:
final_test_table

,data,Target
0,"Premier League, Premier League, League One, Ch...",[soccer.football_league]
1,"Uni-President Lions, Wei Chuan Dragons, Uni-Pr...",[baseball.baseball_team]
2,"Wei Chuan Dragons, Sinon Bulls, Sinon Bulls, U...",[baseball.baseball_team]
3,"Football, Football, Football, Ice hockey, Ice ...",[cvg.cvg_genre]
4,"Joel Skinner, Rick Rodriguez, Gerald Perry",[baseball.baseball_player]
...,...,...
802,"J. League Division 2, J. League Division 2, J....",[soccer.football_league]
803,"J. League Division 1, J. League Division 1, J....",[soccer.football_league]
804,"J. League Division 1, J. League Division 2, J....",[soccer.football_league]
805,"Las Vegas 51s, Jacksonville Suns, Vero Beach D...",[baseball.baseball_team]


In [138]:
final_train_table=pd.DataFrame()
all_values=[]
labelsx=[]
for index_cta, row_cta in cta_train.iterrows():
    

    for table_index,table_row in enumerate(row_cta[6]):
        labels=[]
        for label in  row_cta[7][table_index]:
            if label in cta_all_labels:
                labels.append(label)
        

        if len(labels)>0:
            value_list=[]
            for value_index, value in table_row:
                value_list.append(value[1])
            joined_value_list = ', '.join(map(str, value_list))   
            
            all_values.append(joined_value_list)
            #joined_label=', '.join(map(str, row_cta[7][table_index]))
            joined_label=labels
            labelsx.append(joined_label)

final_train_table['data']=all_values
final_train_table['Target']=labelsx

print(len(all_values))
print(len(labelsx))

38720
38720


In [140]:
final_train_table

,data,Target
0,"Persian Gulf Cup, Persian Gulf Cup, Azadegan L...",[soccer.football_league]
1,"Pro League, Pro League, Pro League, Pro League...",[soccer.football_league]
2,"Ron Villone, Bob Wolcott, Chris Widger, Andy S...",[baseball.baseball_player]
3,"Ligat ha'Al, Ligat ha'Al, Ligat ha'Al, Liga Le...",[soccer.football_league]
4,"Nine Network, ARD, RTL plus, RCTI, RCTI, RCTI,...",[tv.tv_network]
...,...,...
38715,"Peter Gwinn and Rob Dubbin, Peter Gwinn and La...",[tv.tv_writer]
38716,"Argentine Primera, Argentine Primera, Argentin...",[soccer.football_league]
38717,"Seven Network, Dubai TV, ZDF, TVN, NBC",[tv.tv_network]
38718,"737 NAS, 800 NAS, 801 NAS, 809 NAS, 815 NAS, 8...",[military.military_unit]


In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(min_df=3)
X_train = tfidf.fit_transform(final_train_table['data'])
X_test = tfidf.transform(final_test_table['data'])

X_test.shape


(807, 19252)

In [142]:
types =[]
for value in final_train_table['Target']:
    for i in value:
        types.append(i)
types=list(set(types))

indexes=[]
for value in final_train_table['Target']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

final_train_table['encoded_label']=indexes


In [143]:
final_train_table

,data,Target,encoded_label
0,"Persian Gulf Cup, Persian Gulf Cup, Azadegan L...",[soccer.football_league],[1]
1,"Pro League, Pro League, Pro League, Pro League...",[soccer.football_league],[1]
2,"Ron Villone, Bob Wolcott, Chris Widger, Andy S...",[baseball.baseball_player],[4]
3,"Ligat ha'Al, Ligat ha'Al, Ligat ha'Al, Liga Le...",[soccer.football_league],[1]
4,"Nine Network, ARD, RTL plus, RCTI, RCTI, RCTI,...",[tv.tv_network],[36]
...,...,...,...
38715,"Peter Gwinn and Rob Dubbin, Peter Gwinn and La...",[tv.tv_writer],[11]
38716,"Argentine Primera, Argentine Primera, Argentin...",[soccer.football_league],[1]
38717,"Seven Network, Dubai TV, ZDF, TVN, NBC",[tv.tv_network],[36]
38718,"737 NAS, 800 NAS, 801 NAS, 809 NAS, 815 NAS, 8...",[military.military_unit],[30]


In [187]:
valuesx = list(final_train_table['encoded_label'])

find_max=[]
for x in valuesx:
    for i in x:
        find_max.append(i)


n_values = np.max(find_max) + 1
#print(find_max)
target_encoded=[]
for values in valuesx:
    #print(values)
    y_train = [0] * n_values
    for i in values:

        y_train =y_train+ np.eye(n_values)[i]
    target_encoded.append(y_train)
    #print("----------")

In [181]:
len(target_encoded)

38720

In [190]:
len(y_train)

38720

In [189]:
y_train = target_encoded

In [150]:
final_train_table['encoded_matrix']=y_train

In [151]:
final_train_table

,data,Target,encoded_label,encoded_matrix
0,"Persian Gulf Cup, Persian Gulf Cup, Azadegan L...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"Pro League, Pro League, Pro League, Pro League...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"Ron Villone, Bob Wolcott, Chris Widger, Andy S...",[baseball.baseball_player],[4],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Ligat ha'Al, Ligat ha'Al, Ligat ha'Al, Liga Le...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Nine Network, ARD, RTL plus, RCTI, RCTI, RCTI,...",[tv.tv_network],[36],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
38715,"Peter Gwinn and Rob Dubbin, Peter Gwinn and La...",[tv.tv_writer],[11],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38716,"Argentine Primera, Argentine Primera, Argentin...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38717,"Seven Network, Dubai TV, ZDF, TVN, NBC",[tv.tv_network],[36],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38718,"737 NAS, 800 NAS, 801 NAS, 809 NAS, 815 NAS, 8...",[military.military_unit],[30],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [152]:
indexes=[]
for value in final_test_table['Target']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

final_test_table['encoded_label']=indexes

In [153]:
final_test_table

,data,Target,encoded_label
0,"Premier League, Premier League, League One, Ch...",[soccer.football_league],[1]
1,"Uni-President Lions, Wei Chuan Dragons, Uni-Pr...",[baseball.baseball_team],[35]
2,"Wei Chuan Dragons, Sinon Bulls, Sinon Bulls, U...",[baseball.baseball_team],[35]
3,"Football, Football, Football, Ice hockey, Ice ...",[cvg.cvg_genre],[34]
4,"Joel Skinner, Rick Rodriguez, Gerald Perry",[baseball.baseball_player],[4]
...,...,...,...
802,"J. League Division 2, J. League Division 2, J....",[soccer.football_league],[1]
803,"J. League Division 1, J. League Division 1, J....",[soccer.football_league],[1]
804,"J. League Division 1, J. League Division 2, J....",[soccer.football_league],[1]
805,"Las Vegas 51s, Jacksonville Suns, Vero Beach D...",[baseball.baseball_team],[35]


In [177]:
valuesx = list(final_test_table['encoded_label'])

target_encoded=[]
for values in valuesx:
    #print(values)
    y_test = [0] * n_values
    for i in values:

        y_test =y_test+ np.eye(n_values)[i]
    target_encoded.append(y_test)
    #print("----------")



In [178]:
y_test = target_encoded

In [179]:
len(y_test)

807

In [157]:
final_test_table['encoded_matrix']=y_test

In [175]:
final_test_table

,data,Target,encoded_label,encoded_matrix
0,"Premier League, Premier League, League One, Ch...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"Uni-President Lions, Wei Chuan Dragons, Uni-Pr...",[baseball.baseball_team],[35],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"Wei Chuan Dragons, Sinon Bulls, Sinon Bulls, U...",[baseball.baseball_team],[35],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Football, Football, Football, Ice hockey, Ice ...",[cvg.cvg_genre],[34],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Joel Skinner, Rick Rodriguez, Gerald Perry",[baseball.baseball_player],[4],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
802,"J. League Division 2, J. League Division 2, J....",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
803,"J. League Division 1, J. League Division 1, J....",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
804,"J. League Division 1, J. League Division 2, J....",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
805,"Las Vegas 51s, Jacksonville Suns, Vero Beach D...",[baseball.baseball_team],[35],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [159]:
final_train_table

,data,Target,encoded_label,encoded_matrix
0,"Persian Gulf Cup, Persian Gulf Cup, Azadegan L...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"Pro League, Pro League, Pro League, Pro League...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"Ron Villone, Bob Wolcott, Chris Widger, Andy S...",[baseball.baseball_player],[4],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Ligat ha'Al, Ligat ha'Al, Ligat ha'Al, Liga Le...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Nine Network, ARD, RTL plus, RCTI, RCTI, RCTI,...",[tv.tv_network],[36],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
38715,"Peter Gwinn and Rob Dubbin, Peter Gwinn and La...",[tv.tv_writer],[11],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38716,"Argentine Primera, Argentine Primera, Argentin...",[soccer.football_league],[1],"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38717,"Seven Network, Dubai TV, ZDF, TVN, NBC",[tv.tv_network],[36],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38718,"737 NAS, 800 NAS, 801 NAS, 809 NAS, 815 NAS, 8...",[military.military_unit],[30],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [185]:
X_train.shape

(38720, 19252)

In [192]:
len(y_train)

38720

In [161]:
# train_pool = Pool(X_train, y_train)
# test_pool = Pool(X_test, y_test)

In [213]:
clf = CatBoostClassifier(
    loss_function='MultiLogloss',
    eval_metric='HammingLoss',
    iterations=50,
)
clf.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.0275854	total: 13.2s	remaining: 10m 45s
1:	learn: 0.0216286	total: 26.8s	remaining: 10m 43s
2:	learn: 0.0195388	total: 40s	remaining: 10m 27s
3:	learn: 0.0191416	total: 53.2s	remaining: 10m 12s
4:	learn: 0.0183821	total: 1m 7s	remaining: 10m 4s
5:	learn: 0.0176520	total: 1m 21s	remaining: 9m 54s
6:	learn: 0.0174259	total: 1m 34s	remaining: 9m 43s
7:	learn: 0.0166629	total: 1m 48s	remaining: 9m 31s
8:	learn: 0.0164138	total: 2m 2s	remaining: 9m 18s
9:	learn: 0.0157011	total: 2m 16s	remaining: 9m 5s
10:	learn: 0.0153283	total: 2m 30s	remaining: 8m 54s
11:	learn: 0.0148670	total: 2m 46s	remaining: 8m 47s
12:	learn: 0.0144593	total: 3m 1s	remaining: 8m 35s
13:	learn: 0.0143609	total: 3m 15s	remaining: 8m 23s
14:	learn: 0.0140726	total: 3m 30s	remaining: 8m 10s
15:	learn: 0.0137208	total: 3m 47s	remaining: 8m 4s
16:	learn: 0.0135065	total: 4m 3s	remaining: 7m 53s
17:	learn: 0.0132280	total: 4m 18s	remaining: 7m 39s
18:	learn: 0.0131303	total: 4m 35s	rema

In [214]:
y_pred = clf.predict(X_test)

In [215]:
X_test.shape

(807, 19252)

In [216]:
len(y_pred)

807

In [217]:
y_pred[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [219]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       0.97      0.85      0.91       110
           2       1.00      0.44      0.62         9
           3       1.00      0.67      0.80         3
           4       0.84      0.57      0.68       109
           5       1.00      0.50      0.67         2
           6       1.00      0.75      0.86        32
           7       1.00      0.67      0.80         3
           8       0.93      0.68      0.79        57
           9       1.00      0.23      0.38        13
          10       1.00      0.62      0.77        16
          11       0.00      0.00      0.00         1
          12       1.00      0.70      0.82        10
          13       1.00      0.79      0.88        14
          14       0.00      0.00      0.00         0
          15       0.89      0.61      0.72        28
          16       0.00      0.00      0.00         5
    

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [221]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test, y_pred, average="macro")

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [242]:
macro

0.5664897607069851

In [243]:
micro_f1 = f1_score(y_test, y_pred, average="micro")

In [244]:
print(micro_f1)

0.7652548456568556


In [245]:
len(y_pred)

807

In [246]:
y_pred[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [232]:
import pickle
with open('CatBoost Evaluation model/wikiTables-CTA-model.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [235]:
import joblib
joblib.dump(clf, 'CatBoost Evaluation model/wikiTables-CTA-model.joblib')

['CatBoost Evaluation model/wikiTables-CTA-model.joblib']

In [236]:
import catboost
clf.save_model('CatBoost Evaluation model/wikiTables-CTA-model.cbm')


In [237]:
with open('CatBoost Evaluation model/wikiTables-CTA-model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [247]:
loaded_model

WikiTable CPA

In [250]:
cpa_train = pd.read_json('WikiData/train.table_rel_extraction.json')
cpa_test = pd.read_json('WikiData/test.table_rel_extraction.json')

In [251]:
cpa_all_labels = ['time.event.locations', 'music.artist.album', 'sports.sports_team.sport', 'baseball.baseball_team.league', 'tv.tv_program.country_of_origin', 'music.album.artist', 'sports.sports_team.location', 'time.event.instance_of_recurring_event', 'aviation.airline.hubs', 'sports.sports_championship_event.champion', 'sports.sports_facility.teams', 'baseball.baseball_player.position_s', 'sports.sports_league.teams-sports.sports_league_participation.team', 'tv.tv_network.programs-tv.tv_network_duration.program', 'sports.sports_league_season.league', 'olympics.olympic_athlete.country-olympics.olympic_athlete_affiliation.country', 'american_football.football_player.position_s', 'music.composer.compositions', 'meteorology.tropical_cyclone.tropical_cyclone_season', 'cvg.computer_videogame.developer', 'tv.tv_character.appeared_in_tv_program-tv.regular_tv_appearance.actor', 'cvg.computer_videogame.publisher', 'soccer.football_player.position_s', 'tv.tv_program.original_network-tv.tv_network_duration.network', 'music.composition.composer', 'ice_hockey.hockey_player.hockey_position', 'book.author.works_written', 'film.film.genre', 'film.film.directed_by', 'film.film.produced_by', 'film.film.language', 'broadcast.broadcast.area_served', 'award.award_category.category_of', 'location.location.nearby_airports', 'location.country.official_language']

In [253]:
cpa_test

,0,1,2,3,4,5,6,7
0,670791-1,yukon legislative assembly,670791,current members,,"[name, party]","[[[[0, 0], [33384061, 'Kevin Barr']], [[1, 0],...",[[government.politician.party-government.polit...
1,37797825-1,list of songs recorded by led zeppelin,37797825,,references,"[song, writer(s)]","[[[[0, 0], [1727257, '"" Achilles Last Stand ""'...",[[music.composition.composer]]
2,33863640-1,list of air india fc managers,33863640,statistics,,"[name, nationality]","[[[[0, 0], [39919400, 'Bimal Ghosh']], [[2, 0]...","[[people.person.nationality, people.person.pla..."
3,56602-2,astana,56602,sport,,"[club, sport, league]","[[[[0, 0], [21491213, 'Astana F.C.']], [[1, 0]...","[[sports.sports_team.sport], [sports.sports_te..."
4,29014813-1,northwestern mexico,29014813,states,,"[state, capital, largest city]","[[[[0, 0], [199793, 'Baja California']], [[1, ...",[[location.administrative_division.capital-loc...
...,...,...,...,...,...,...,...,...
1462,19783157-1,list of house members of the 40th parliament o...,19783157,alberta,,"[name, electoral district]","[[[[0, 0], [2266595, 'Lee Richardson']], [[1, ...",[[government.politician.government_positions_h...
1463,1276543-4,balaji telefilms,1276543,film productions,,"[title, director(s)]","[[[[0, 0], [10140052, 'Kyo Kii... Main Jhuth N...","[[film.film.directed_by, award.award_nominated..."
1464,23000314-2,list of fantasy films of the 2010s,23000314,forthcoming,,"[title, director, cast]","[[[[2, 0], [29769606, '47 Ronin']], [[3, 0], [...",[[award.award_nominated_work.award_nominations...
1465,808980-1,list of rulers of austria,808980,house of babenberg,,"[name, birth, marriages, death]","[[[[0, 0], [1100652, 'Leopold I the Illustriou...","[[people.person.parents], [people.person.spous..."


In [258]:
test_examples = []

for primary_row_index, primary_row_values in cpa_test.iterrows():
    new_col_labels = []
    for col_index, col_labels in enumerate(primary_row_values[7]):
        sampled_labels = []
        for label in col_labels:
            if label in cpa_all_labels:
                sampled_labels.append(label)
        
        if len(sampled_labels) !=0:
            new_col_labels.append(sampled_labels)

            column_string = ""

            for row_index in primary_row_values[6][0][:5]:
                column_string += f"{row_index[1][1]} "
            column_string = column_string.strip()

            column_string += ""
        
            for row_index in primary_row_values[6][col_index+1][:5]:
                column_string += f"{row_index[1][1]} "
            column_string = column_string.strip()

            test_examples.append([primary_row_values[0], col_index, column_string, sampled_labels])

In [259]:
test_examples

[['37797825-1',
  0,
  '" Achilles Last Stand " " All My Love " " Babe I\'m Gonna Leave You " " Baby Come On Home " " Black Country Woman "Jimmy Page , Robert Plant John Paul Jones , Plant Anne Bredon , Page, Plant Bert Berns , Page, Plant John Bonham',
  ['music.composition.composer']],
 ['56602-2',
  0,
  'Astana F.C. Bayterek F.C. F.C. Astana-1964 Astana Pro Team Astana TigersFootball Football Football Ice hockey Ice hockey',
  ['sports.sports_team.sport']],
 ['36890482-1',
  0,
  '2007–08 2008–09 2009–10 2010–11 2011–12Pro League Pro League Division 1 Division 1 Division 1',
  ['time.event.instance_of_recurring_event']],
 ['57447-3',
  0,
  'Carolina Panthers Charlotte Bobcats Note A Charlotte Hounds Charlotte Checkers Charlotte KnightsFootball Basketball Lacrosse Ice hockey Baseball',
  ['sports.sports_team.sport']],
 ['2576653-1',
  0,
  'The Farm of Seven Sins Bibi Fricotin Le Chevalier de la nuit Love in a Hot Climate Marie Antoinette Queen of FranceMarcel Blistène Robert Darèn

In [260]:
final_table_cpa=pd.DataFrame()
labels=[]
values=[]
for i in test_examples:
    labels.append(i[-1])
    values.append(i[2])

final_table_cpa['data']=values
final_table_cpa['cpa label']=labels

    

In [257]:
train_table_cpa=final_table_cpa
train_table_cpa=train_table_cpa

In [261]:
test_table_cpa=final_table_cpa
test_table_cpa=test_table_cpa

In [262]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(min_df=3)
X_train_cpa = tfidf.fit_transform(train_table_cpa['data'])
X_test_cpa = tfidf.transform(test_table_cpa['data'])

X_test_cpa.shape

(535, 16276)

In [263]:
X_train_cpa.shape

(18340, 16276)

In [264]:
types =[]
for value in train_table_cpa['cpa label']:
    for i in value:
        types.append(i)
types=list(set(types))

indexes=[]
for value in train_table_cpa['cpa label']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

train_table_cpa['encoded_label']=indexes

In [265]:
valuesx = list(train_table_cpa['encoded_label'])

find_max=[]
for x in valuesx:
    for i in x:
        find_max.append(i)


n_values = np.max(find_max) + 1
#print(find_max)
target_encoded=[]
for values in valuesx:
    #print(values)
    y_train = [0] * n_values
    for i in values:

        y_train =y_train+ np.eye(n_values)[i]
    target_encoded.append(y_train)
    #print("----------")



In [266]:
y_train_cpa=target_encoded

In [267]:
y_train_cpa

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.]),
 arr

In [268]:
indexes=[]
for value in test_table_cpa['cpa label']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

test_table_cpa['encoded_label']=indexes

valuesx = list(test_table_cpa['encoded_label'])

target_encoded=[]
for values in valuesx:
    #print(values)
    y_test = [0] * n_values
    for i in values:

        y_test =y_test+ np.eye(n_values)[i]
    target_encoded.append(y_test)
    #print("----------")

In [269]:
y_test_cpa=target_encoded

In [167]:
#train_pool = Pool(X_train_cpa, y_train_cpa)
#test_pool = Pool(X_test_cpa, y_test_cpa)

In [275]:
clf_cpa = CatBoostClassifier(
    loss_function='MultiLogloss',
    eval_metric='HammingLoss',
    iterations=50,
)
clf_cpa.fit(X_train_cpa, y_train_cpa)

Learning rate set to 0.5
0:	learn: 0.0286415	total: 7.39s	remaining: 6m 1s
1:	learn: 0.0257937	total: 12.6s	remaining: 5m 2s
2:	learn: 0.0240520	total: 17.9s	remaining: 4m 41s
3:	learn: 0.0232186	total: 23.2s	remaining: 4m 27s
4:	learn: 0.0217916	total: 28.6s	remaining: 4m 17s
5:	learn: 0.0213834	total: 33.9s	remaining: 4m 8s
6:	learn: 0.0207602	total: 40.4s	remaining: 4m 8s
7:	learn: 0.0203443	total: 47s	remaining: 4m 6s
8:	learn: 0.0199704	total: 53.2s	remaining: 4m 2s
9:	learn: 0.0196542	total: 59.1s	remaining: 3m 56s
10:	learn: 0.0191697	total: 1m 5s	remaining: 3m 50s
11:	learn: 0.0182754	total: 1m 11s	remaining: 3m 46s
12:	learn: 0.0179545	total: 1m 18s	remaining: 3m 42s
13:	learn: 0.0176788	total: 1m 25s	remaining: 3m 38s
14:	learn: 0.0173734	total: 1m 31s	remaining: 3m 34s
15:	learn: 0.0166319	total: 1m 38s	remaining: 3m 29s
16:	learn: 0.0163920	total: 1m 44s	remaining: 3m 23s
17:	learn: 0.0162346	total: 1m 51s	remaining: 3m 17s
18:	learn: 0.0159931	total: 1m 56s	remaining: 3m 1

In [276]:
y_pred_cpa = clf_cpa.predict(X_test_cpa)

In [277]:
accuracy = eval_metric(y_test_cpa, y_pred_cpa, 'Accuracy')[0]
print(f'Accuracy: {accuracy}')

Accuracy: 0.6299065420560748


In [278]:
accuracy_per_class = eval_metric(y_test_cpa, y_pred_cpa, 'Accuracy:type=PerClass')
for cls, value in zip(clf.classes_, accuracy_per_class):
    print(f'Accuracy for class {cls}: {value}')

Accuracy for class 0: 0.9757009345794393
Accuracy for class 1: 1.0
Accuracy for class 2: 0.9775700934579439
Accuracy for class 3: 0.9981308411214953
Accuracy for class 4: 0.9906542056074766
Accuracy for class 5: 0.9850467289719627
Accuracy for class 6: 0.9813084112149533
Accuracy for class 7: 0.9962616822429906
Accuracy for class 8: 0.9962616822429906
Accuracy for class 9: 0.9869158878504672
Accuracy for class 10: 0.9925233644859813
Accuracy for class 11: 0.9869158878504672
Accuracy for class 12: 0.9813084112149533
Accuracy for class 13: 0.9906542056074766
Accuracy for class 14: 1.0
Accuracy for class 15: 0.9925233644859813
Accuracy for class 16: 1.0
Accuracy for class 17: 0.9906542056074766
Accuracy for class 18: 0.9813084112149533
Accuracy for class 19: 0.9869158878504672
Accuracy for class 20: 0.9850467289719627
Accuracy for class 21: 0.9962616822429906
Accuracy for class 22: 0.9962616822429906
Accuracy for class 23: 0.994392523364486
Accuracy for class 24: 0.9887850467289719
Accura

In [ ]:
for metric in ('Precision', 'Recall', 'F1'):
    print(metric)
    values = eval_metric(y_test_cpa, y_pred_cpa, metric)
    for cls, value in zip(clf.classes_, values):
        print(f'class={cls}: {value:.4f}')
    print()

In [279]:
print("Classification Report:")
print(classification_report(y_test_cpa, y_pred_cpa))

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       1.00      1.00      1.00         2
           2       1.00      0.29      0.45        17
           3       0.95      1.00      0.98        20
           4       1.00      0.17      0.29         6
           5       1.00      0.38      0.56        13
           6       0.57      0.36      0.44        11
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         2
           9       1.00      0.12      0.22         8
          10       1.00      0.86      0.92        28
          11       1.00      0.76      0.86        29
          12       1.00      0.29      0.44        14
          13       1.00      0.38      0.55         8
          14       1.00      1.00      1.00         3
          15       1.00      0.33      0.50         6
          16       1.00      1.00      1.00         1
    

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [280]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test_cpa, y_pred_cpa, average="macro")

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [282]:
macro

0.5662751481255126

In [283]:
micro_f1 = f1_score(y_test_cpa, y_pred_cpa, average="micro")

In [284]:
print(micro_f1)

0.7575757575757577


In [285]:
len(y_pred_cpa)

535

In [288]:
y_pred_cpa[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [289]:
import pickle
with open('CatBoost Evaluation model/wikiTables-CPA-model.pkl', 'wb') as file:
    pickle.dump(clf_cpa, file)